# Mobile App Data Project

For this project I am going to analyze a dataset based on the mobile app store to decide what type of app will make the most money by analyzing ratings, free vs not free, age restrictions, genre

## Goal

Make a general guess on the best kind of app to make

In [1]:
from csv import reader

### Google Play data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### App Store data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
print(android_header)
print('\n')
explore_data(android, 0, 4, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13


There are 13 columns and 10841 apps.  Important columns could include 'App', 'Category', 'Rating', 'Type', 'Price' and 'Genres'

In [4]:
print(ios_header)
print('\n')
explore_data(ios,0,4, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


16 columns and 7197 apps. 

Important columns: 'track name', 'price', 'user_rating' and 'prime_genre'

##### Documentation for Data Sets
[Google Play Data Set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

[IOS App Store Data Set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [5]:
print(android[10472]) #possible example of error in data entry
print('\n')
print(android_header)
print('\n')
print(android[0]) #comparison to correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


We see that the 2nd column 'Category' is missing, shifting everything to the left so this entry needs to be deleted

In [6]:
del(android[10472]) #DONT RUN THIS AGAIN OR YOU'LL DELETE THE NEXT ROW

In [7]:
print(android[10472]) #check to see if row has been deleted

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


## Removing Duplicate Entries

Per Kaggle discussion, the google play data set has many duplicate entries that we will need to delete.  To find the duplicates, we will iterate over the android data set to find non unique app names and put them in a seperate list, then count how many duplicates there are. 

In [8]:
duplicate_apps = []
unique_apps = []
for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Duplicate count', len(duplicate_apps))
print('\n')
print('Unique count', len(unique_apps))

Duplicate count 1181


Unique count 9659


To choose the entry we want to keep, we look at the number of reviews in each entry, the one with the highest number is the most recent entry so we will keep that one and delete all entries with lower reviews.

We will do this by creating a dictionary with keys as unique app name, coorsponding key value as the highest number of reviews of that app

In [9]:
reviews_max = {}

for entry in android:
    name = entry[0]
    n_reviews = float(entry[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print(len(reviews_max)) #should be 9659

9659


The purpose of `if name in reviews_max and reviews_max[name] < n_reviews:reviews_max[name] = n_reviews` is to update the dictionary key only if the number of reviews are more than the existing entry.

In [10]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) # this needs to be in block for some reason

explore_data(android_clean, 0, 7, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+

## Removing Non-English Apps

We are going to remove every app with a name containing a symbol that is not in the English alphabet. English text is contained in ord() 0-127 so we will remove anything >127

In [11]:
def eng_check(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(eng_check('Instagram'))
print(eng_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(eng_check('Docs To Go™ Free Office Suite'))
print(eng_check('Instachat 😜'))

True
False
False
False


We see that the tm and emoji characters don't pass the english test, however we don't want to lose these so we will only remove the app if there are more than three non-ASCII characters.

In [12]:
def eng_check(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True


print(eng_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(eng_check('Docs To Go™ Free Office Suite'))
print(eng_check('Instachat 😜'))

#note that the >3 code should remain outside the for loop.  
#This has something to do with the return function inside a for loop

False
True
True


In [13]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if eng_check(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if eng_check(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

## Isolate Free apps

In [14]:
free_android = []
free_ios = []

for app in android_english:
    price = app[7]
    if price == '0':
        free_android.append(app)

for app in ios_english:
    price = app[4]
    if price == '0.0':  #make sure this is 0.0, not 0
        free_ios.append(app)

print(len(free_android))
print(len(free_ios))

8864
3222


## Most Common Apps by Genre

In order to lower risk and overhead cost, we will first want to release a barebones android app, and add it to the google play store.  If the app has good response, develop it further.  If profitable, then create an ios version and add to the apple store.  

To get a sense of what type of app to build, we will make a frequency table of genres out of the free, english app list we created

Category and Genre indexes resp. for android: [1] and [9]
Genre for ios: [11]

In [15]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [19]:
display_table(free_ios, 11) # prime genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The most common genre is games, followed by entertainment and photo & video.
It appears that most apps are designed for fun and entertainment.  Based off this frequency table, I would assume that the entertainment apps are fully saturated by companies with more resources than mine.  A large number of apps for a particular genre does imply a large number of users, or that that genre of app is easy to develop at low cost. 

In [20]:
display_table(free_android, 1) #category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [21]:
display_table(free_android, 9) #genre

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In the google play store, we see that productivity and tools are far more common compared to entertainment apps.  Once again, we are not sure if these have the most users so further analysis is required. 

In [37]:
ftable = freq_table(free_ios, 11)

for genre in ftable:
    total = 0
    len_genre = 0
    for app in free_ios:
        genre_app = app[11]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_rating=(total/len_genre)
    print(genre, ':', int(avg_rating))

Catalogs : 4004
Medical : 612
Productivity : 21028
Reference : 74942
Book : 39758
Social Networking : 71548
Utilities : 18684
Photo & Video : 28441
Weather : 52279
Food & Drink : 33333
News : 21248
Lifestyle : 16485
Navigation : 86090
Travel : 28243
Shopping : 26919
Finance : 31467
Games : 22788
Entertainment : 14029
Music : 57326
Sports : 23008
Health & Fitness : 23298
Business : 7491
Education : 7003


High number of ratings include: social networking, weather, navigation, music and reference.  I recommend staying away from social networking and navigation, as we do not want to compete with facebook, ig, maps or waze.  Reference apps have to do with dictionary.com and the bible so this is an outlier.  I would recommend looking into the practical, health and fitness or photo and video genre's.  These seem the least saturated with a high amount of use reviews.

In [41]:
ftableandroid = freq_table(free_android, 1)

for category in ftableandroid:
    total = 0
    len_category = 0
    for app in free_android:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+','')
            n_installs = n_installs.replace(',','')
            total += float(n_installs)
            len_category += 1
            
    avg_installs = total / len_category
    print(category, ':', int(avg_installs))

BOOKS_AND_REFERENCE : 8767811
HEALTH_AND_FITNESS : 4188821
BUSINESS : 1712290
PERSONALIZATION : 5201482
ART_AND_DESIGN : 1986335
TOOLS : 10801391
AUTO_AND_VEHICLES : 647317
SOCIAL : 23253652
VIDEO_PLAYERS : 24727872
COMICS : 817657
PARENTING : 542603
NEWS_AND_MAGAZINES : 9549178
LIBRARIES_AND_DEMO : 638503
DATING : 854028
ENTERTAINMENT : 11640705
LIFESTYLE : 1437816
FOOD_AND_DRINK : 1924897
WEATHER : 5074486
GAME : 15588015
MEDICAL : 120550
PHOTOGRAPHY : 17840110
MAPS_AND_NAVIGATION : 4056941
SHOPPING : 7036877
TRAVEL_AND_LOCAL : 13984077
SPORTS : 3638640
FINANCE : 1387692
BEAUTY : 513151
PRODUCTIVITY : 16787331
HOUSE_AND_HOME : 1331540
FAMILY : 3695641
EDUCATION : 1833495
COMMUNICATION : 38456119
EVENTS : 253542


We see that communication apps have the most installs, however this is dominated by whatsapp, fb messenger, google and skype.  Other interesting genres with high installs include: business, tools, food and drink, game, photography, traveling and local, productivity.  We remember from earlier that the productivity apps were the most saturatred while entertainment was not (in the google play store).  If we focus on a genre with high installs and low frequency, maybe we will have a better chance to make money.  Therefore, I recommend looking at food and drink or photography apps.

In [44]:
for app in free_android:
    if app[1] == 'PHOTOGRAPHY' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

B612 - Beauty & Filter Camera : 100,000,000+
YouCam Makeup - Magic Selfie Makeovers : 100,000,000+
Sweet Selfie - selfie camera, beauty cam, photo edit : 100,000,000+
Google Photos : 1,000,000,000+
Retrica : 100,000,000+
Photo Editor Pro : 100,000,000+
BeautyPlus - Easy Photo Editor & Selfie Camera : 100,000,000+
PicsArt Photo Studio: Collage Maker & Pic Editor : 100,000,000+
Photo Collage Editor : 100,000,000+
Z Camera - Photo Editor, Beauty Selfie, Collage : 100,000,000+
PhotoGrid: Video & Pic Collage Maker, Photo Editor : 100,000,000+
Candy Camera - selfie, beauty camera, photo editor : 100,000,000+
YouCam Perfect - Selfie Photo Editor : 100,000,000+
Camera360: Selfie Photo Editor with Funny Sticker : 100,000,000+
S Photo Editor - Collage Maker , Photo Collage : 100,000,000+
AR effect : 100,000,000+
Cymera Camera- Photo Editor, Filter,Collage,Layout : 100,000,000+
LINE Camera - Photo editor : 100,000,000+
Photo Editor Collage Maker Pro : 100,000,000+


With the exception of google photos, none of these apps pop out to me as having dominated the market.  It seems like you could make money with filter packs or some type of photo editor.  